<a href="https://colab.research.google.com/github/staceysv/tttc-light-js/blob/main/examples/T3C_LLM_Pipeline_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Talk to the City (T3C) Workflow

Talk to the City (T3C) summarizes and organizes diverse human perspectives for easier analysis and decision-making.
This notebook documents and visualizes the LLM inteface, factoring out the prompts & processing and logging everything, including intermediate stages and costs, to W&B.

## LLM Prompting Pipeline

1. Given all comments, create a taxonomy tree of concise topics + subtopics.
2. For each comment, extract claims and assign them to a subtopic in the taxonomy tree.
3. (no LLM calls) Order the subtopics by the most claims.
4. Deduplicate claims in each subtopic.

### More context
* [T3C Github Repo](https://github.com/AIObjectives/tttc-light-js)
* [T3C Product Overview](https://ai.objectives.institute/talk-to-the-city)

### Sample CSV data

* Tweets on AI safety: 100, 500, 1000, 2893
* Reddit climate change: posts titles 100, 250, 500
* Goodreads poetry book reviews, 500




# 0 Setup & imports

## 0.0 Import packages, auth with W&B + OAI

You'll need a [W&B API key](https://www.wandb.ai/authorize) and an OpenAI key. This colab will not log or store the keys.

In [ ]:
!pip install openai
!pip install -qqq weave
!pip install -qqq wandb
import wandb
import pandas as pd

wandb.login()

In [ ]:
import os
# authenticate with OpenAI
from getpass import getpass

if os.getenv("OPENAI_API_KEY") is None:
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

## 0.1 Utils

Helpful functions

In [ ]:
from datetime import datetime
from pytz import timezone
import pytz

def time_here():
  date_format='%m/%d/%Y %H:%M:%S'
  date = datetime.now()
  date = date.astimezone(timezone('US/Pacific'))
  return date.strftime(date_format)

def topic_tree(taxonomy):
  core_tree = []
  full_tree = taxonomy["taxonomy"]
  for main_topic in full_tree:
    topic = main_topic["topicName"]
    desc = main_topic["topicShortDescription"]
    subtopic_list = []
    for subtopic in main_topic["subtopics"]:
      if "subtopicName" in subtopic:
        sub_topic = subtopic["subtopicName"]
      else:
        print("WARNING: NO TOPIC NAME")
        continue
      if "subtopicShortDescription" in subtopic:
        sub_desc = subtopic["subtopicShortDescription"]
      else:
        print("WARNING: NO TOPIC DESCRIPTION")
        sub_desc = "N/A"
      subtopic_list.append({sub_topic : sub_desc})
    core_tree.append({ topic : desc, "subtopic" : subtopic_list})
  return core_tree

def cute_print(json_obj):
  """Returns a pretty version of a dictionary as properly-indented and scaled
  json in html for at-a-glance review in W&B"""
  str_json = json.dumps(json_obj, indent=1)
  cute_html = '<pre id="json"><font size=2>' + str_json + "</font></pre>"
  return wandb.Html(cute_html)



## 0.2 Input data

Sample lists of comments which might be useful for testing/future adhoc exploration. Load directly from code or CSV file.

Testing factors to consider:
* duplicates: very similar/identical statements within a topic or across themes/topics
* subject/object, main point, length and clarity
* intensity/generality of opinions

### 0.2.0 Load comments from CSV

Send the "comments" column from a dataframe through the pipeline and save the source data to W&B.


In [ ]:
CSV_FILENAME = "../tw_1000.csv"
df = pd.read_csv(open(CSV_FILENAME, 'r'))
comments = df["comments"]

# optionally upload to W&B
#wandb.init(project=WB_PROJECT_NAME, name="upload_csv_comments", group="csv_comment_upload")
#wandb.log({"tiny_dream_test" : df})
#wandb.run.finish()

### 0.2.1 Load tiny test lists from code

In [ ]:
# past sample comments possibly useful for testing
MVP_TEST = ["I love cats", "I really really love dogs", "I'm not sure about birds"]
#comments = ["none"]
MVP_TEST.extend(["Cats are my favorite", "Dogs are the best", "No seriously dogs are great", "Birds I'm hesitant about", "Cats can be walked outside and they don't have to", "Dogs need to be walked regularly, every day", "Dogs can be trained to perform adorable moves on verbal command", "Can cats be trained?", "Dogs and cats are both adorable and fluffy", "Good pets are chill", "Cats are fantastic", "A goldfish is my top choice"])
SCIFI_TEST = ["My favorite fantasy novel is Name of the Wind", "Terra Ignota is the best scifi series of all time", "Idk about Kim Stanley Robinson"]
SCIFI_TEST.extend(["Name of the Wind is predictable and hard to read", "Some of Kim Stanley Robinson is boring", "Terra Ignota gets slow in the middle and hard to follow",
            "Ada Palmer is spectacular", "Becky Chambers has fantastic aliens in her work", "Ministry for the Future and Years of Rice and Salt are really comprehensive and compelling stories",
            "Do we still talk about Lord of the Rings or Game of Thrones or is epic fantasy over", "What about Ted Chiang he is so good", "Greg Egan is really good at characters and plot and hard science",
            "I never finished Accelerando", "Ministry for the Future is about the climate transition", "The climate crisis is a major theme in Ministry for the Future", "Ministry for the Future is about climate"])

MVP_TEST.extend(["Lizards are scary", "Kittens are my favorite when they have snake-like scales", "Hairless cats are unique", "Flying lizards are majestic", "Kittens are so boring"])

# *************************
# SET OR ADD COMMENTS HERE
# *************************
comments = SCIFI_TEST

In [ ]:
print("Total comments: ", len(comments))

## 0.3 Prompts

All the prompts for the pipeline

In [ ]:
SYS_PROMPT = """
You are a professional research assistant. You have helped run many public consultations,
surveys and citizen assemblies. You have good instincts when it comes to extracting interesting insights.
You are familiar with public consultation tools like Pol.is and you understand the benefits
for working with very clear, concise claims that other people would be able to vote on.
"""

COMMENT_TO_TREE_PROMPT = """
I will give you a list of comments.
Please propose a way to organize the information contained in these comments into topics and subtopics of interest.
Keep the topic and subtopic names very concise and use the short description to explain what the topic is about.

Return a JSON object of the form {
  "taxonomy": [
    {
      "topicName": string,
      "topicShortDescription": string,
      "subtopics": [
        {
          "subtopicName": string,
          "subtopicShortDescription": string,
        },
        ...
      ]
    },
    ...
  ]
}
Now here is the list of comments:
"""

COMMENT_TO_CLAIMS = """
I'm going to give you a comment made by a participant and a list of topics and subtopics which have already been extracted.
I want you to extract a list of concise claims that the participant may support.
We are only interested in claims that can be mapped to one of the given topic and subtopic.
The claim must be fairly general but not a platitude.
It must be something that other people may potentially disagree with. Each claim must also be atomic.
For each claim, please also provide a relevant quote from the transcript.
The quote must be as concise as possible while still supporting the argument.
The quote doesn't need to be a logical argument.
It could also be a personal story or anecdote illustrating why the interviewee would make this claim.
You may use "[...]" in the quote to skip the less interesting bits of the quote.
/return a JSON object of the form {
  "claims": [
    {
      "claim": string, // a very concise extracted claim
      "quote": string // the exact quote,
      "topicName": string // from the given list of topics
      "subtopicName": string // from the list of subtopics
    },
    // ...
  ]
}

Now here is the list of topics/subtopics:"""
# also include in prompt:
# append ${taxonomy}
# comments: And then here is the comment:"""

DEDUP_PROMPT = """
I'm going to give you a JSON object containing a list of claims with some ids.
I want you to remove any near-duplicate claims from the list by nesting some claims under some top-level claims.
For example, if we have 5 claims and claim 3 and 5 are similar to claim 2, we will nest claim 3 and 5 under claim 2.
The nesting will be represented as a JSON object where the keys are the ids of the
top-level claims and the values are lists of ids of the nested claims.

Return a JSON object of the form {
  "nesting": {
    "claimId1": [],
    "claimId2": ["claimId3", "claimId5"],
    "claimId4": []
  }
}

And now, here are the claims:"""

# 1: Configure pipeline run

W&B variables for convenience:
* set RUN_NAME for each new pass through the pipeline
* optionally set EXP_GROUP for each new set of experiments (easier to toggle visibility/metrics by new logic/day/coding session/etc)



In [ ]:
RUN_NAME = "tw_424"
EXP_GROUP = "uc"
WB_PROJECT_NAME = "t3c_pipeline"

In [ ]:
# maps to gpt-4-0125-preview - token costs:
# $10/1M in, $30/1M out = $0.1/10K in, $0.3/10K out
MODEL = "gpt-4-turbo-preview"
COST_IN_PER_10K = 0.1
COST_OUT_PER_10K = 0.3

# periodically update from W&B
AVG_TREE_LEN_TOKS = 614
AVG_CLAIM_TOKS_OUT = 130
AVG_TOPIC_COUNT = 12
AVG_DEDUP_INPUT_TOK = 12
AVG_DEDUPED_CLAIMS_FACTOR = 0.6

guess_cost = 0
actual_cost = 0

# log sheet
#name,group,time,rows,chars,guess_cost,actual_cost,1_cost,2_cost,4_cost,num_themes,num_topics,num_claims
#eventually we want these more..granular/averaged?

# 2 Estimate costs

Estimate costs for each step and overall:
* 1: SYS_PROMPT + COMMENTS_TO_TREE_PROMPT + len(comments)
* 2: count(comments) * (SYS_PROMPT + COMMENT_TO_CLAIMS_PROMPT + taxonomy output of 1) + len(comments)
* 3: (SYS_PROMPT + DEDUP_PROMPT) * len(topics) where > 1 claim

1 token ~= 4 characters of text for common English text. This translates to roughly ¾ of a word (so 100 tokens ~= 75 words).

In [ ]:
# estimate cost before completing template
comments_total = sum([len(c) for c in comments])
N_sys_prompt = len(SYS_PROMPT)

# set topic count to be square root of comment length?

# inputs only
step_1_cost = (N_sys_prompt + len(COMMENT_TO_TREE_PROMPT) + comments_total) / 4.0
step_2_cost = (comments_total/4.0) + len(comments) * (((N_sys_prompt + len(COMMENT_TO_CLAIMS))/ 4.0) + AVG_TREE_LEN_TOKS)
step_4_cost = ((N_sys_prompt + len(DEDUP_PROMPT)) /4.0) * (len(comments) ** 0.33) * AVG_DEDUP_INPUT_TOK
total_cost = step_1_cost + step_2_cost + step_4_cost

# then we divide by 10K and multiply by constants

print(step_1_cost)
print(step_2_cost)
print(step_4_cost)
print(total_cost)

cost_in = (total_cost * COST_IN_PER_10K) / 10000.0

# outputs — how do we estimate this?
step_1_out_cost = AVG_TREE_LEN_TOKS
step_2_out_cost = len(comments) * AVG_CLAIM_TOKS_OUT
step_4_out_cost = len(comments) * AVG_DEDUPED_CLAIMS_FACTOR

cost_out = step_1_out_cost + step_2_out_cost + step_4_out_cost
print(step_1_out_cost)
print(step_2_out_cost)
print(step_4_out_cost)
print(cost_out)

total_cost_out = (cost_out * COST_OUT_PER_10K) / 10000.0
print("IN: ", cost_in, " OUT: ", total_cost_out)
net_cost = cost_in + total_cost_out
print("TOTAL GUESS: $", net_cost)
guess_cost = net_cost

# 3: Run pipeline

## Step 1: Comments to tree

Given the full list of comments, call LLM to create a taxonomy of main topics (themes) and subtopics with short descriptions.

In [ ]:
import json
from openai import OpenAI
import wandb
import weave
import json

weave.init(WB_PROJECT_NAME)
wandb.init(project = WB_PROJECT_NAME, name=RUN_NAME, group=EXP_GROUP,
           # TODO: add more config here
           config={"model" : MODEL,
                   "$_in_10K" : COST_IN_PER_10K,
                   "$_out_10K" : COST_OUT_PER_10K,
                   "cost_guess" : guess_cost
                  })

# track token counts+costs for pipeline
TK_TOT = 0
TK_IN = 0
TK_OUT = 0
NUM_THEMES = 0

@weave.op()
def comments_to_tree(comments:list)-> dict:
    client = OpenAI()

    # append comments to prompt
    full_prompt = COMMENT_TO_TREE_PROMPT
    for comment in comments:
      full_prompt += "\n" + comment

    response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": SYS_PROMPT
        },
        {
            "role": "user",
            "content": full_prompt
        }
        ],
        temperature=0.0,
        response_format={ "type": "json_object" }
    )
    tree = response.choices[0].message.content
    return {"tree" : json.loads(tree), "usage" : response.usage}

# estimate cost before completing template
comment_lengths = [len(c) for c in comments]
wandb.log({"comm_N" : len(comments), "comm_bins" : comment_lengths})

with weave.attributes({"model" : MODEL, "stage" : "1_comments_to_tree"}):
  resp = comments_to_tree(comments)
  taxonomy = resp["tree"]
  usage = resp["usage"]
  print(taxonomy)
  print(usage)
  NUM_THEMES = len(taxonomy["taxonomy"])

# in case comments are empty / for W&B Table logging
comment_list = "none"
if len(comments) > 1:
  comment_list = "\n".join(comments)
tl = [[comment_list, cute_print(topic_tree(taxonomy)), json.dumps(taxonomy,indent=1)]]

# update token counts
TK_TOT += usage.total_tokens
TK_IN += usage.prompt_tokens
TK_OUT += usage.completion_tokens

# calculate and log actual cost here?
actual_1_cost = (COST_IN_PER_10K * TK_IN + COST_OUT_PER_10K * TK_OUT) / 10000.0
print("actual$:", actual_1_cost)
actual_cost += actual_1_cost

wandb.log({
    "u/1/N_tok": usage.total_tokens,
    "u/1/in_tok" : usage.prompt_tokens,
    "u/1/out_tok": usage.completion_tokens,
    "u/1/cost" : actual_1_cost,
    "u/N/N_tok" : TK_TOT,
    "u/N/in_tok": TK_IN,
    "u/N/out_tok" : TK_OUT,
    "u/N/cost" : actual_cost,
    "rows_to_tree" : wandb.Table(data=tl, columns = ["comments", "taxonomy", "raw_llm_out"])
})


## Step 2: One comment > extract claims

For each comment, extract claims and assign to a specific subtopic in the given taxonomy.

In [ ]:
weave.init(WB_PROJECT_NAME)

@weave.op()
def comment_to_claims(comment:str)-> dict:
    client = OpenAI()

    # add taxonomy and comment to prompt template
    full_prompt = COMMENT_TO_CLAIMS
    taxonomy_string = json.dumps(taxonomy, indent=1)
    full_prompt += "\n" + taxonomy_string + "\nAnd then here is the comment:\n" + comment

    response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": SYS_PROMPT
        },
        {
            "role": "user",
            "content": full_prompt
        }
        ],
        temperature=0.0,
        response_format={ "type": "json_object" }
    )
    claims = response.choices[0].message.content
    return {"claims" : json.loads(claims), "usage" : response.usage}

# track totals for this pipeline step only
# what do we want to know for this step?
# we want the individual counts of each
TK_2_IN = 0
TK_2_OUT = 0
TK_2_TOT = 0

c2c = []
c2c_html = []
with weave.attributes({
    "model" : MODEL, "stage" : "2_comment_to_claims", "run" : RUN_NAME}):
  for comment in comments:
    resp = comment_to_claims(comment)
    claims = resp["claims"]
    usage = resp["usage"]
    print(comment)
    print(claims)
    c2c.append(claims)

    # format for logging to W&B
    viz_claims = cute_print(claims)
    c2c_html.append([comment, viz_claims, json.dumps(claims,indent=1)])

    TK_2_IN += usage.prompt_tokens
    TK_2_OUT += usage.completion_tokens
    TK_2_TOT += usage.total_tokens

    TK_TOT += usage.total_tokens
    TK_IN += usage.prompt_tokens
    TK_OUT += usage.completion_tokens


    # update per-comment tokens
    wandb.log({
      "u/2/s_N_tok": usage.total_tokens,
      "u/2/s_in_tok" : usage.prompt_tokens,
      "u/2/s_out_tok": usage.completion_tokens,
      "u/2/t_N_tok": TK_2_TOT,
      "u/2/t_in_tok" : TK_2_IN,
      "u/2/t_out_tok": TK_2_OUT
    })

# calculate and log actual cost here?
actual_2_cost = (COST_IN_PER_10K * TK_2_IN + COST_OUT_PER_10K * TK_2_OUT) / 10000.0
print("actual$:", actual_2_cost)
actual_cost += actual_2_cost

wandb.log({
    # update total token counts
    "u/N/N_tok" : TK_TOT,
    "u/N/in_tok": TK_IN,
    "u/N/out_tok" : TK_OUT,
    "u/N/cost" : actual_cost,
    "u/2/cost" : actual_2_cost,
    "row_to_claims" : wandb.Table(data=c2c_html, columns = ["comments", "claims", "raw_llm_out"])})



## Step 3: [nonLLM] Count + sort by claims DESC

Sort the taxonomy by the number of claims in each subtopic, desc.
Note that this pipeline stage doesn't call any LLMs.

In [ ]:
NUM_TOPICS = 0
NUM_CLAIMS = 0

def sort_taxonomy(tree, c2c):
  node_counts = {}
  # count number of claims in each topic node of the outline
  for cmt, cmt_claims in zip(comments, c2c):
    for claim in cmt_claims["claims"]:
      if claim["topicName"] in node_counts:
        node_counts[claim["topicName"]]["total"] += 1
        if claim["subtopicName"] in node_counts[claim["topicName"]]["subtopics"]:
          node_counts[claim["topicName"]]["subtopics"][claim["subtopicName"]]["total"] += 1
          node_counts[claim["topicName"]]["subtopics"][claim["subtopicName"]]["claims"].append(claim["claim"])
        else:
          node_counts[claim["topicName"]]["subtopics"][claim["subtopicName"]] = { "total" : 1, "claims" : [claim["claim"]]}
      else:
        node_counts[claim["topicName"]] = {"total" : 1, "subtopics" : {claim["subtopicName"] : {"total" : 1, "claims" : [claim["claim"]]}}}
  return node_counts

# log sorted taxonomy
sorted_taxonomy = sort_taxonomy(taxonomy, c2c)
print(sorted_taxonomy)
html_data = [[cute_print(sorted_taxonomy), json.dumps(sorted_taxonomy, indent=1)]]

for theme, topics in sorted_taxonomy.items():
  for theme_key, topic_details in topics.items():
    if theme_key == "subtopics":
        for topic_key, claim_details in topic_details.items():
            NUM_CLAIMS += claim_details["total"]
        NUM_TOPICS += len(topic_details)

print(NUM_TOPICS)
print(NUM_CLAIMS)

wandb.log({"sort_tree" : wandb.Table(data=html_data, columns = ["sorted_taxonomy", "raw_llm_output"])})

In [ ]:
for k, v in sorted_taxonomy.items():
  for i, j in v.items():
    if i == "subtopics":
      print(j.keys())

## Step 4: Dedup claims in each subtopic

Find similar claims in the list for each subtopic. This logic could be much cleaner.

In [ ]:
weave.init(WB_PROJECT_NAME)

@weave.op()
def dedup_claims(claims:str)-> dict:
    client = OpenAI()

    # add claims with enumerated ids
    full_prompt = DEDUP_PROMPT
    for i, rc in enumerate(claims):
      full_prompt += "\nclaimId"+str(i)+ ": " + rc

    response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": SYS_PROMPT
        },
        {
            "role": "user",
            "content": full_prompt
        }
        ],
        temperature=0.0,
        response_format={ "type": "json_object" }
    )
    deduped_claims = response.choices[0].message.content
    return {"dedup_claims" : json.loads(deduped_claims), "usage" : response.usage}

# we want the individual counts of each
TK_4_IN = 0
TK_4_OUT = 0
TK_4_TOT = 0

print(c2c)
nested_claims = {}
dupe_counts = {}
with weave.attributes({"model" : MODEL, "stage" : "4_dedup_claims", "run" : RUN_NAME}):
  tl_data = []
  for topic, subt in sorted_taxonomy.items():
    for sub_topic, subtd in subt["subtopics"].items():
      print("num claims: ", len(subtd["claims"]))
      # don't dedup solo claims
      if len(subtd["claims"]) > 1:
        resp = dedup_claims(subtd["claims"])
        deduped_claims = resp["dedup_claims"]
        usage = resp["usage"]

        # let's check if they're duplicated?
        # this is harder than we thought!
        has_dupes = False
        if "nesting" in deduped_claims:
          for claim_key, claim_vals in deduped_claims["nesting"].items():
            if len(claim_vals) > 0:
              has_dupes = True
              # extract index...
              ckey = int(claim_key[-1:])
              dupe_keys = [int(c_key[-1:]) for c_key in claim_vals]
              dupe_counts[subtd["claims"][ckey]] = dupe_keys

        # for logging to wandb
        tl_data.append(["\n".join(subtd["claims"]), cute_print(deduped_claims), json.dumps(deduped_claims, indent=1)])

        # append dupe claims & filter
        if has_dupes:
          nested_claims[sub_topic] = {"dupes" : deduped_claims, "og" : subtd["claims"]}
        wandb.log({
            "u/4/s_N_tok": usage.total_tokens,
            "u/4/s_in_tok" : usage.prompt_tokens,
            "u/4/s_out_tok": usage.completion_tokens,
            "u/4/t_N_tok": TK_4_TOT,
            "u/4/t_in_tok" : TK_4_IN,
            "u/4/t_out_tok": TK_4_OUT
        })
        TK_4_TOT += usage.total_tokens
        TK_4_IN += usage.prompt_tokens
        TK_4_OUT += usage.completion_tokens
        TK_TOT += usage.total_tokens
        TK_IN += usage.prompt_tokens
        TK_OUT += usage.completion_tokens

  # calculate and log actual cost here?
  actual_4_cost = (COST_IN_PER_10K * TK_4_IN + COST_OUT_PER_10K * TK_4_OUT) / 10000.0
  print("actual$:", actual_4_cost)
  actual_cost += actual_4_cost

  wandb.log({
      "u/N/N_tok" : TK_TOT,
      "u/N/in_tok": TK_IN,
      "u/N/out_tok" : TK_OUT,
      "u/4/cost" : actual_4_cost,
      "u/N/cost" : actual_cost,
      "dedup_subclaims" : wandb.Table(data=tl_data, columns = ["sub_claim_list", "deduped_claims", "raw_llm_output"])})


print(json.dumps(dupe_counts, indent=2))

# 4: Save Approximate T3C Report

Merge duplicate claims and log a simplified T3C report to W&B.

In [ ]:
def synth_t3c_report(ttree, dupes):
  ltree = {}
  for theme, theme_d in ttree.items():
    theme_total = 0
    topic_list = {}
    for topic, topic_d in theme_d["subtopics"].items():
      theme_total += topic_d["total"]
      if topic in nested_claims:
        # this one has some dupes
        # for each duplicate claim, we list the duplicate ids
        # but we don't currently merge them as "similar claims"
        rerank = {}
        for c in topic_d["claims"]:
          if c in dupe_counts:
            new_label = c + " (" + str(len(dupe_counts[c]) + 1) + "x:"
            for ckey in dupe_counts[c]:
              new_label += " " + str(ckey) + ","
            new_label += ")"
            rerank[new_label] = len(dupe_counts[c])
          else:
            rerank[c] = 0
        ranked = sorted(rerank.items(), key=lambda x: x[1], reverse=True)
        new_claims = [r[0] for r in ranked]
        print(new_claims)
        topic_list[topic] = {"total" : topic_d["total"], "claims" : new_claims}
      else:
        topic_list[topic] = {"total" : topic_d["total"], "claims" : topic_d["claims"]}

    # sort topics
    sorted_topics = sorted(topic_list.items(), key=lambda x: x[1]["total"], reverse=True)

    ltree[theme] = {"total" : theme_total, "topics" : sorted_topics}

  # sort full tree
  sorted_tree = sorted(ltree.items(), key=lambda x: x[1]["total"], reverse=True)
  return sorted_tree

ltree = synth_t3c_report(sorted_taxonomy, dupe_counts)
print(json.dumps(ltree, indent=4))

# log sorted taxonomy
html_data = [[cute_print(ltree), json.dumps(ltree, indent=1)]]

# log final costs
total_run_cost = TK_IN * (COST_IN_PER_10K/10000.0) + TK_OUT * (COST_OUT_PER_10K/10000.0)
print("guessed: ", guess_cost)
print("total from TK: ", total_run_cost)
print("total from math: ", actual_cost)

# TODO: print cost here!!!
# log sheet
#name,group,time,rows,chars,guess_cost,tok_cost,actual_cost,1_cost,2_cost,4_cost,num_themes,num_topics,num_claims
#eventually we want these more..granular/averaged?

log_row = [RUN_NAME, EXP_GROUP, time_here(),len(comments),comments_total,round(guess_cost,2),
           round(total_run_cost, 2),round(actual_cost,2),round(actual_1_cost,2),round(actual_2_cost,2),round(actual_4_cost,2),
           NUM_THEMES, NUM_TOPICS, NUM_CLAIMS ]
csv_log = ",".join([str(x) for x in log_row])
print(csv_log)

wandb.log({
    "cost/tok_total" :  total_run_cost,
    "cost/actual" : actual_cost,
    "cost/1": actual_1_cost,
    "cost/2" : actual_2_cost,
    "cost/4" : actual_4_cost,
    "csv_log" : csv_log,
    "t3c_report" : wandb.Table(data=html_data, columns = ["t3c_report", "raw_llm_output"])})


# done with this run!
wandb.run.finish()

# Appendix

## Dev TODOs
- log raw dictionaries as well as nicely-formatted json so we can download & post-process later
- we need to sum the stage 2 token counts

- we need to count / estimate / track tree size


- condense prompts
- enable cost logging
- enable different models

- compare swapping long/short description names in prompt
- add unit tests based on comments
- double-check/improve model config logging in W&B runs
- check: allow for empty comments, w/o meaningful claims
- track finish_reason = completion.choices[0].finish_reason

